In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy.spatial
import pandas as pd
import sklearn.decomposition
import matplotlib.pyplot as plt
# import keras
from sklearn import preprocessing
from sklearn.metrics import pairwise_distances,mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
from utils.readProfiles import *
from utils.pred_models import *
from utils.saveAsNewSheetToExistingFile import saveAsNewSheetToExistingFile
from multiprocessing import Pool

sns.set_style("whitegrid")
# from utils import networksEvol, tsne, readProfiles
import umap

## Load data

- CDRP-BBBC047-Bray-CP-GE (Cell line: U2OS):
    * There are 30,430 and 21,782 unique compounds for CP and GE datasets, respectively.
    * Median number of replicates for each dataset is as follows: CP: 4 (approx), GE: 3 (approx). 
    * 20,358 compounds are present in both datasets.
    * Replicate Level Shapes (nSamples x nFeatures): cp:  (153386 , 1783) ,  ge:  (68120 , 977)
    * Treatment Level Shapes (nSamples x nFeatures): cp:  (30430, 1786)   ,  ge:  (21782, 981) 
    * Merged Profiles Shape:                              (20358, 2766)
    * High Rep corr: CP: 30618 to 7892, l1k: 21069 to 2870, overlap:  3
    
    
- CDRPBIO-BBBC036-Bray-CP-GE (Cell line: U2OS):
    * There are 30,430 and 21,782 unique compounds for CP and GE datasets, respectively.
    * Median number of replicates for each dataset is as follows: CP: 4 (approx), GE: 3 (approx). 
    * 20,358 compounds are present in both datasets.
    * Replicate Level Shapes (nSamples x nFeatures): cp:  (153386 , 1783) ,  ge:  (68120 , 977)
    * Treatment Level Shapes (nSamples x nFeatures): cp:  (30430, 1786)   ,  ge:  (21782, 981) 
    * Merged Profiles Shape:                   
    * High Rep corr: CP: 2239 to 312, l1k: 1535 to 448, overlap:  131


- LUAD-BBBC041-Caicedo-CP-GE (Cell line: A549) : 
    * There are 593 and 529 unique alleles for CP and GE datasets, respectively.
    * Median number of replicates for each dataset is as follows: CP: 8 (approx), GE: 8 (approx).
    * 525 alleles are present in both datasets.
    * Replicate Level Shapes (nSamples x nFeatures): cp:  (6144 , 1783) ,  ge:  (4232 , 978)
    * Treatment Level Shapes (nSamples x nFeatures): cp: (593, 1786) , ge: (529, 979) 
    * Merged Profiles Shape:                             (525, 2764)    
    * High Rep corr: CP: x to x, l1k: x to x, overlap:  x    
    
    
- TA-ORF-BBBC037-Rohban-CP-GE (Cell line: U2OS) :
    * There are 299 and 226 number of unique compounds for CP and GE datasets respectively.
    * Median number of replicates for each dataset is as follows: CP: 5 (approx), GE: 2 (approx).
    * 188 alleles are present in both datasets.
    * Replicate Level Shapes (nSamples x nFeatures):         cp: (1920 , 1783) ,  ge: (729 , 978)
    * Treatment Level Shapes (nSamples x nFeatures+metadata):cp: (323, 1784)   ,  ge: (328, 979)
    * Merged Profiles Shape:                                     (149, 2762)
    * High Rep corr: CP: x to x, l1k: x to x, overlap:  x    
    
    
- LINCS-Pilot1-CP-GE (Cell line: A549) :
    * There are 1570 unique compounds across 7 doses for CP dataset
    * There are x unique compounds across 7 doses for GE dataset
    * Median number of replicates for each dataset is as follows: CP: 5 (approx), GE: 3 (approx).
    * 6984 "compounds-dose" are present in both datasets. 
    * Replicate Level Shapes (nSamples x nFeatures):         cp: (52223 , 1747) ,  ge: (27837 , 978)
    * Treatment Level Shapes (nSamples x nFeatures+metadata):cp: (9394, 1748)   ,  ge: (8370, 979)
    * Merged Profiles Shape:                                     (6984, 2726)
    * High Rep corr: CP: 9394 to 4647, l1k: 8369  to  2338, overlap:  1140
    

#### Paths

In [ ]:
# procProf_dir='/home/ubuntu/datasetsbucket/Rosetta-GE-CP/'
# procProf_dir='/home/ubuntu/bucket/projects/2018_04_20_Rosetta/workspace/'
procProf_dir = "../../../rosetta/broad/workspace/"
# metadata_dir='/home/ubuntu/bucket/projects/2018_04_20_Rosetta/workspace/metadata/'
results_dir = "./results/"


## Prediction of single GE expression levels based on the full CP profiles

In [ ]:
################################################
# dataset options: 'CDRP' , 'LUAD', 'TAORF', 'LINCS', 'CDRP-bio'
datasets = ["LUAD"]
# datasets=['LINCS', 'CDRP-bio','CDRP'];
# datasets=['TAORF','LUAD','LINCS', 'CDRP-bio']

DT_kfold = {"LUAD": 10, "TAORF": 5, "LINCS": 25, "CDRP-bio": 6, "CDRP": 40}

################################################
# CP Profile Type options: 'augmented' , 'normalized', 'normalized_variable_selected'
profileType = "normalized_variable_selected"

################################################
# filtering to compounds which have high replicates for both GE and CP datasets
# highRepOverlapEnabled=0
# 'highRepUnion','highRepOverlap'
filter_perts = "highRepOverlap"

################################################
pertColName = "PERT"


if filter_perts:
    f = "filt"
else:
    f = ""

# def f(dataset):
for dataset in datasets:

    (
        mergProf_treatLevel,
        cp_features,
        l1k_features,
    ) = read_paired_treatment_level_profiles(
        procProf_dir, dataset, profileType, filter_perts, 1
    )

    l1k = mergProf_treatLevel[[pertColName] + l1k_features]
    cp = mergProf_treatLevel[[pertColName] + cp_features]

    if dataset == "LINCS":
        cp["Compounds"] = cp["PERT"].str[0:13]
        l1k["Compounds"] = l1k["PERT"].str[0:13]
    else:
        cp["Compounds"] = cp["PERT"]
        l1k["Compounds"] = l1k["PERT"]

    le = preprocessing.LabelEncoder()
    group_labels = le.fit_transform(l1k["Compounds"].values)

    scaler_ge = preprocessing.StandardScaler()
    scaler_cp = preprocessing.StandardScaler()
    l1k_scaled = l1k.copy()
    l1k_scaled[l1k_features] = scaler_ge.fit_transform(l1k[l1k_features].values)
    cp_scaled = cp.copy()
    cp_scaled[cp_features] = scaler_cp.fit_transform(
        cp[cp_features].values.astype("float64")
    )

    for model in ["Lasso"]:  # ["Lasso","MLP","RFR"]:

        if 1:
            cp = cp_scaled.copy()
            l1k = l1k_scaled.copy()

        ##############################

        k_fold = DT_kfold[dataset]

        pred_df = pd.DataFrame(index=range(k_fold), columns=l1k_features)
        pred_df_rand = pd.DataFrame(index=range(k_fold), columns=l1k_features)
        ii = 0
        for l in l1k_features:
            ii += 1
            print(ii)
            if model == "Lasso":
                scores, scores_rand = lasso_cv_plus_model_selection(
                    cp[cp_features], l1k[l], k_fold, group_labels, 1
                )
            #                 sfaadadd
            #                 scores,scores_rand=lasso_cv(cp[cp_features],l1k[l],k_fold,group_labels)
            elif model == "MLP":
                scores, scores_rand = MLP_cv_plus_model_selection(
                    cp[cp_features], l1k[l], k_fold, group_labels, 1
                )
            #                 scores,scores_rand=MLP_cv(cp[cp_features],l1k[l],k_fold,group_labels)
            elif model == "Ridge":
                scores, scores_rand = ridge_cv_plus_model_selection(
                    cp[cp_features], l1k[l], k_fold, group_labels, 1
                )
            #                 gafgfdssgfd
            #                 scores,scores_rand=MLP_cv(cp[cp_features],l1k[l],k_fold,group_labels)
            elif model == "RFR":
                scores, scores_rand = RFR_cv_plus_model_selection(
                    cp[cp_features], l1k[l], k_fold, group_labels, 1
                )

            pred_df[l] = scores
            pred_df_rand[l] = scores_rand

        ########################### mapping prob_ids to genes names
        pred_df, _ = rename_affyprobe_to_genename(pred_df, l1k_features)
        pred_df_rand, _ = rename_affyprobe_to_genename(pred_df_rand, l1k_features)

        meltedPredDF = pd.melt(pred_df).rename(
            columns={"variable": "lmGens", "value": "pred score"}
        )
        meltedPredDF_rand = pd.melt(pred_df_rand).rename(
            columns={"variable": "lmGens", "value": "pred score"}
        )
        meltedPredDF["d"] = "n-folds"
        meltedPredDF_rand["d"] = "random"
        filename = results_dir + "/SingleGenePred/scores.xlsx"

        profTypeAbbrev = "".join([s[0] for s in profileType.split("_")])

#         saveAsNewSheetToExistingFile(filename,pd.concat([meltedPredDF,meltedPredDF_rand],ignore_index=True),\
#                                      model+'-'+dataset+'-'+profTypeAbbrev+'-'+f+'-'+str(k_fold)+'-ht')


In [ ]:
# l1k_features

## Prediction of single CP features based on the full GE profiles

In [ ]:
################################################
# dataset options: 'CDRP' , 'LUAD', 'TAORF', 'LINCS', 'CDRP-bio'
datasets = ["LINCS", "CDRP-bio"]
# datasets=['TAORF','CDRP'];

DT_kfold = {"LUAD": 10, "TAORF": 10, "LINCS": 20, "CDRP-bio": 10, "CDRP": 40}

# DT_kfold={'LUAD':9, 'TAORF':5, 'LINCS':25, 'CDRP-bio':6,'CDRP':40}


################################################
# CP Profile Type options: 'augmented' , 'normalized', 'normalized_variable_selected'
# 'normalized_feature_select_dmso'
profileType = "normalized"

################################################
# filtering to compounds which have high replicates for both GE and CP datasets
# highRepOverlapEnabled=0
# 'highRepUnion','highRepOverlap'
filter_perts = "highRepOverlap"


################################################
pertColName = "PERT"

if filter_perts:
    f = "filt"
else:
    f = ""

# def f(dataset):
for dataset in datasets:

    (
        mergProf_treatLevel,
        cp_features,
        l1k_features,
    ) = read_paired_treatment_level_profiles(
        procProf_dir, dataset, profileType, filter_perts, 1
    )

    l1k = mergProf_treatLevel[[pertColName] + l1k_features]
    cp = mergProf_treatLevel[[pertColName] + cp_features]

    if dataset == "LINCS":
        cp["Compounds"] = cp["PERT"].str[0:13]
        l1k["Compounds"] = l1k["PERT"].str[0:13]
    else:
        cp["Compounds"] = cp["PERT"]
        l1k["Compounds"] = l1k["PERT"]

    le = preprocessing.LabelEncoder()
    group_labels = le.fit_transform(l1k["Compounds"].values)

    scaler_ge = preprocessing.StandardScaler()
    scaler_cp = preprocessing.StandardScaler()
    l1k_scaled = l1k.copy()
    l1k_scaled[l1k_features] = scaler_ge.fit_transform(l1k[l1k_features].values)
    cp_scaled = cp.copy()
    cp_scaled[cp_features] = scaler_cp.fit_transform(
        cp[cp_features].values.astype("float64")
    )

    for model in ["Lasso", "MLP"]:

        #         if model=="MLP":
        #             cp_scaled[cp_features] =preprocessing.MinMaxScaler(feature_range=(-1, 1)).fit_transform(cp_scaled[cp_features].values)
        #             cp_scaled[l1k_features] =preprocessing.MinMaxScaler(feature_range=(-1, 1)).fit_transform(cp_scaled[l1k_features].values)

        if 1:
            cp = cp_scaled.copy()
            l1k = l1k_scaled.copy()

        ##############################

        k_fold = DT_kfold[dataset]
        #         k_fold=int(np.unique(group_labels).shape[0]/20)
        pred_df = pd.DataFrame(index=range(k_fold), columns=cp_features)
        pred_df_rand = pd.DataFrame(index=range(k_fold), columns=cp_features)
        ii = 0
        for c in cp_features:
            ii += 1
            print(ii)
            if model == "Lasso":
                scores, scores_rand = lasso_cv_plus_model_selection(
                    l1k[l1k_features], cp[c], k_fold, group_labels, 1
                )
            elif model == "MLP":
                scores, scores_rand = MLP_cv_plus_model_selection(
                    l1k[l1k_features], cp[c], k_fold, group_labels, 1
                )
            pred_df[c] = scores
            pred_df_rand[c] = scores_rand

        meltedPredDF = pd.melt(pred_df).rename(
            columns={"variable": "CP-Features", "value": "pred score"}
        )
        meltedPredDF_rand = pd.melt(pred_df_rand).rename(
            columns={"variable": "CP-Features", "value": "pred score"}
        )
        meltedPredDF["d"] = "n-folds"
        meltedPredDF_rand["d"] = "random"
        #         filename='../../results/SingleCPfeatPred/scores.xlsx'
        filename = results_dir + "/SingleCPfeatPred/scores.xlsx"

        profTypeAbbrev = "".join([s[0] for s in profileType.split("_")])

        saveAsNewSheetToExistingFile(
            filename,
            pd.concat([meltedPredDF, meltedPredDF_rand], ignore_index=True),
            model
            + "-"
            + dataset
            + "-"
            + profTypeAbbrev
            + "-"
            + f
            + "-"
            + str(k_fold)
            + "-ht",
        )


#     return

# with Pool(10) as p:
#     p.map(f, datasets)


### CP Category specific scores for single gene prediction - based on Lasso  regression

In [ ]:
# %%capture
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning

# simplefilter("ignore", category=ConvergenceWarning)
# from sklearn.exceptions import ConvergenceWarning
ConvergenceWarning("ignore")

################################################
# dataset options: 'CDRP' , 'LUAD', 'TAORF', 'LINCS', 'CDRP-bio'
# datasets=['LUAD', 'TAORF', 'LINCS', 'CDRP-bio'];
datasets = ["LUAD"]

# DT_kfold={'LUAD':10, 'TAORF':5, 'LINCS':20, 'CDRP-bio':20,'CDRP':40}

DT_kfold = {"LUAD": 9, "TAORF": 5, "LINCS": 25, "CDRP-bio": 6, "CDRP": 40}


################################################
# CP Profile Type options: 'augmented' , 'normalized', 'normalized_variable_selected'
# 'normalized_feature_select_dmso'
profileType = "normalized"

################################################
# filtering to compounds which have high replicates for both GE and CP datasets
# highRepOverlapEnabled=0
# 'highRepUnion','highRepOverlap'
filter_perts = "highRepOverlap"


################################################
pertColName = "PERT"
profileLevel = "treatment"
#'replicate'  or  'treatment'
if filter_perts:
    f = "filt"
else:
    f = ""

# def f(dataset):
for dataset in datasets:

    (
        mergProf_treatLevel,
        cp_features,
        l1k_features,
    ) = read_paired_treatment_level_profiles(
        procProf_dir, dataset, profileType, filter_perts, 1
    )

    l1k = mergProf_treatLevel[[pertColName] + l1k_features]
    cp = mergProf_treatLevel[[pertColName] + cp_features]

    if dataset == "LINCS":
        cp["Compounds"] = cp["PERT"].str[0:13]
        l1k["Compounds"] = l1k["PERT"].str[0:13]
    else:
        cp["Compounds"] = cp["PERT"]
        l1k["Compounds"] = l1k["PERT"]

    le = preprocessing.LabelEncoder()
    group_labels = le.fit_transform(l1k["Compounds"].values)

    scaler_ge = preprocessing.StandardScaler()
    scaler_cp = preprocessing.StandardScaler()
    l1k_scaled = l1k.copy()
    l1k_scaled[l1k_features] = scaler_ge.fit_transform(l1k[l1k_features].values)
    cp_scaled = cp.copy()
    cp_scaled[cp_features] = scaler_cp.fit_transform(
        cp[cp_features].values.astype("float64")
    )

    if 1:
        cp = cp_scaled.copy()
        l1k = l1k_scaled.copy()

    ##############################

    # k_fold=10
    k_fold = DT_kfold[dataset]
    Channelss = ["DNA", "RNA", "AGP", "Mito", "ER"]
    featureGroups = ["Texture", "Intensity", "RadialDistribution"]
    relationMat_mpCat = pd.DataFrame(index=l1k_features)
    for ch in range(len(Channelss)):
        for f in range(len(featureGroups)):
            print(ch, f)
            selectedCols = cp.columns[
                cp.columns.str.contains(Channelss[ch])
                & cp.columns.str.contains(featureGroups[f])
                & cp.columns.str.contains("Cells_|Cytoplasm_|Nuclei_")
            ].tolist()
            if selectedCols != []:
                for l in l1k_features:
                    #                     scores,scores_rand=lasso_cv_plus_model_selection(cp[selectedCols],l1k[l],k_fold,group_labels,0)
                    scores, scores_rand = MLP_cv_plus_model_selection(
                        cp[selectedCols], l1k[l], k_fold, group_labels, 0
                    )

                    relationMat_mpCat.loc[
                        l, Channelss[ch] + "_" + featureGroups[f]
                    ] = np.median(scores)

    Channelss = ["Nuclei", "Cytoplasm", "Cells"]
    featureGroups = ["AreaShape"]

    for ch in range(len(Channelss)):
        for f in range(len(featureGroups)):
            selectedCols = cp.columns[
                cp.columns.str.contains(Channelss[ch])
                & cp.columns.str.contains(featureGroups[f])
                & cp.columns.str.contains("Cells_|Cytoplasm_|Nuclei_")
            ].tolist()
            if selectedCols != []:
                for l in l1k_features:
                    #                     scores,scores_rand=lasso_cv_plus_model_selection(cp[selectedCols],l1k[l],k_fold,group_labels,0)
                    scores, scores_rand = MLP_cv_plus_model_selection(
                        cp[selectedCols], l1k[l], k_fold, group_labels, 0
                    )

                    relationMat_mpCat.loc[
                        l, Channelss[ch] + "_" + featureGroups[f]
                    ] = np.median(scores)

    ########################### mapping prob_ids to genes names
    meta = pd.read_csv(
        "../affy_probe_gene_mapping.txt",
        delimiter="\t",
        header=None,
        names=["probe_id", "gene"],
    )
    meta_gene_probID = meta.set_index("probe_id")
    d = dict(zip(meta_gene_probID.index, meta_gene_probID["gene"]))
    relationMat_mpCat = relationMat_mpCat.rename(index=d)

    filename = results_dir + "/SingleGenePred_cpCategoryMap/cat_scores_maps.xlsx"

    profTypeAbbrev = "".join([s[0] for s in profileType.split("_")])

    saveAsNewSheetToExistingFile(
        filename, relationMat_mpCat, dataset + "-" + str(k_fold) + "-MLP-ht"
    )


In [ ]:
# cp.columns[cp.columns.str.contains('RadialDistribution')]

In [ ]:
# topGenes_mito_radial.sort_values(by='Mito_RadialDistribution').index.values

## Leave dataset out Cross validation - on LUAD and LINCS (2-folds)

In [ ]:
################################################
# dataset options: 'CDRP' , 'LUAD', 'TAORF', 'LINCS', 'CDRP-bio'
datasets = ["LUAD", "LINCS"]
# datasets=['LINCS', 'CDRP-bio','CDRP'];

################################################
# CP Profile Type options: 'augmented' , 'normalized', 'normalized_variable_selected'
profileType = "normalized"

################################################
# filtering to compounds which have high replicates for both GE and CP datasets
# highRepOverlapEnabled=0
# 'highRepUnion','highRepOverlap'
filter_perts = "highRepOverlap"

################################################
pertColName = "PERT"
profileLevel = "treatment"
#'replicate'  or  'treatment'
if filter_perts:
    f = "filt"
else:
    f = ""


(
    mergProf_treatLevel_LI,
    cp_features_LI,
    l1k_features_LI,
) = read_paired_treatment_level_profiles(
    procProf_dir, "LINCS", profileType, filter_perts, 1
)

(
    mergProf_treatLevel_LU,
    cp_features_LU,
    l1k_features_LU,
) = read_paired_treatment_level_profiles(
    procProf_dir, "LUAD", profileType, filter_perts, 1
)


In [ ]:
print(
    len(cp_features_LU),
    len(cp_features_LI),
    len(set(cp_features_LU) & set(cp_features_LI)),
)
print(
    len(l1k_features_LU),
    len(l1k_features_LI),
    len(set(l1k_features_LU) & set(l1k_features_LI)),
)


In [ ]:
cp_features_unionFs = list(set(cp_features_LU).union(set(cp_features_LI)))
l1k_features_unionFs = list(set(l1k_features_LU).union(set(l1k_features_LI)))


In [ ]:
len(cp_features_unionFs)

In [ ]:
cp_features_allOverlappingFs = list(
    set(cp_features_LU) & set(cp_features_LI) & set(cp_features_unionFs)
)
l1k_features_allOverlappingFs = list(
    set(l1k_features_LU) & set(l1k_features_LI) & set(l1k_features_unionFs)
)

len(cp_features_allOverlappingFs)
# cp_features_LU

In [ ]:
%%capture
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor

# cp_features=list(set(cp_features_LU) & set(cp_features_LI))
# l1k_features=list(set(l1k_features_LU) & set(l1k_features_LI))
# profileType='normalized'

cp_features=list(set(cp_features_LU) & set(cp_features_LI) & set(cp_features_unionFs))
l1k_features=list(set(l1k_features_LU) & set(l1k_features_LI)& set(l1k_features_unionFs))
profileType='normalized_variable_selected_union'


####################
l1k_LU=mergProf_treatLevel_LU[[pertColName]+l1k_features]
cp_LU=mergProf_treatLevel_LU[[pertColName]+cp_features]

l1k_LI=mergProf_treatLevel_LI[[pertColName]+l1k_features]
cp_LI=mergProf_treatLevel_LI[[pertColName]+cp_features]

####################


l1k_scaled_LU=l1k_LU.copy();l1k_scaled_LI=l1k_LI.copy()
l1k_scaled_LU[l1k_features] = preprocessing.StandardScaler().fit_transform(l1k_LU[l1k_features].values)
l1k_scaled_LI[l1k_features] = preprocessing.StandardScaler().fit_transform(l1k_LI[l1k_features].values)

cp_scaled_LU=cp_LU.copy();cp_scaled_LI=cp_LI.copy()
cp_scaled_LU[cp_features] = preprocessing.StandardScaler().fit_transform(cp_LU[cp_features].values.astype('float64'))
cp_scaled_LI[cp_features] = preprocessing.StandardScaler().fit_transform(cp_LI[cp_features].values.astype('float64'))

fold1=[[cp_scaled_LU,l1k_scaled_LU],[cp_scaled_LI,l1k_scaled_LI]]
fold2=[[cp_scaled_LI,l1k_scaled_LI],[cp_scaled_LU,l1k_scaled_LU]]


pred_df=pd.DataFrame(index=range(2),columns=l1k_features)
pred_df_rand=pd.DataFrame(index=range(2),columns=l1k_features)

# for dt in [fold1,fold2]:
for n,dt in zip([0,1],[fold1,fold2]):
    
    for model in ["Ridge"]:#["Lasso","MLP","Ridge"]:    

        ##############################
        ii=0
        for l in l1k_features:
            ii+=1
            print(ii)

            X_train, X_test = dt[0][0][cp_features].values, dt[1][0][cp_features].values
            y_train, y_test = dt[0][1][l].values, dt[1][1][l]#.values

            if model=="Lasso":
    #             scores,scores_rand=lasso_cv_plus_model_selection(cp[cp_features],l1k[l],k_fold,group_labels)        
    # #                 scores,scores_rand=lasso_cv(cp[cp_features],l1k[l],k_fold,group_labels)
                alphas1 = np.linspace(0, 0.2, 20)
                alphas2 = np.linspace(0.2, 0.5, 10)[1:]
                alphas=np.concatenate((alphas1,alphas2))
            #     alphas = np.logspace(-4, -0.5, 30)
                lasso_cv = linear_model.LassoCV(alphas=alphas, random_state=0, max_iter=1000,selection='random')

                lasso_cv.fit(X_train, y_train)  
                r2_score=lasso_cv.score(X_test, y_test.values) 
                r2_rand=lasso_cv.score(X_test, y_test.sample(frac = 1).values)

            if model=="Ridge":
    #             scores,scores_rand=lasso_cv_plus_model_selection(cp[cp_features],l1k[l],k_fold,group_labels)        
    # #                 scores,scores_rand=lasso_cv(cp[cp_features],l1k[l],k_fold,group_labels)
                alphas = np.linspace(0.1, 0.2, 20)
            #     alphas = np.logspace(-4, -0.5, 30)
                ridge_cv = linear_model.RidgeCV(alphas=alphas)

                ridge_cv.fit(X_train, y_train)  
                r2_score=ridge_cv.score(X_test, y_test.values) 
                r2_rand=ridge_cv.score(X_test, y_test.sample(frac = 1).values)
                
            elif model=="MLP":
#                 scores,scores_rand=MLP_cv_plus_model_selection(cp[cp_features],l1k[l],k_fold,group_labels)
    #                 scores,scores_rand=MLP_cv(cp[cp_features],l1k[l],k_fold,group_labels)  
                mlp_gs = MLPRegressor(activation='logistic',max_iter=500)
                parameter_space = {
                    'hidden_layer_sizes': [(50,),(10,30,10),(50,10),(50,10,10)],
                    'alpha': [0.0001, 0.05,0.01,0.2],
                    'early_stopping':[True,False]
                }

                clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=2)
                clf.fit(X_train, y_train)  
                r2_score=clf.score(X_test, y_test.values)    
                r2_rand=clf.score(X_test, y_test.sample(frac = 1).values)
    
            pred_df.loc[n,l]=r2_score
            pred_df_rand.loc[n,l]=r2_rand

    
########################### mapping prob_ids to genes names    
meta=pd.read_csv("../affy_probe_gene_mapping.txt",delimiter="\t",header=None, names=["probe_id", "gene"])
meta_gene_probID=meta.set_index('probe_id')
d = dict(zip(meta_gene_probID.index, meta_gene_probID['gene']))
pred_df = pred_df.rename(columns=d)    
pred_df_rand = pred_df_rand.rename(columns=d)  

pred_df.loc[0,'DT']='LUAD-LINCS'
pred_df.loc[1,'DT']='LINCS-LUAD'

pred_df_rand.loc[0,'DT']='LUAD-LINCS'
pred_df_rand.loc[1,'DT']='LINCS-LUAD'

meltedPredDF=pd.melt(pred_df,id_vars='DT').rename(columns={'variable':'lmGens','value':'pred score'})
meltedPredDF_rand=pd.melt(pred_df_rand,id_vars='DT').rename(columns={'variable':'lmGens','value':'pred score'})
meltedPredDF['d']="n-folds"
meltedPredDF_rand['d']="random"

filename=results_dir+'/SingleGenePred/scores_cross_dts.xlsx'

profTypeAbbrev=''.join([s[0] for s in profileType.split('_')])

saveAsNewSheetToExistingFile(filename,pd.concat([meltedPredDF,meltedPredDF_rand],ignore_index=True),\
                             model+'-'+profTypeAbbrev+'-'+f+'-ht')
        

In [ ]:
pred_df_rand

In [ ]:
filename = results_dir + "/SingleGenePred/scores_cross_dts.xlsx"

profTypeAbbrev = "".join([s[0] for s in profileType.split("_")])

saveAsNewSheetToExistingFile(
    filename,
    pd.concat([meltedPredDF], ignore_index=True),
    model + "-" + profTypeAbbrev + "-" + f + "-ht",
)


## Leave dataset out Cross validation - on CDRP-bio and LINCS (2-folds)

In [ ]:
################################################
# dataset options: 'CDRP' , 'LUAD', 'TAORF', 'LINCS', 'CDRP-bio'
datasets = ["CDRP-bio", "LINCS"]
# datasets=['LINCS', 'CDRP-bio','CDRP'];

################################################
# CP Profile Type options: 'augmented' , 'normalized', 'normalized_variable_selected'
profileType = "normalized"

################################################
# filtering to compounds which have high replicates for both GE and CP datasets
# highRepOverlapEnabled=0
# 'highRepUnion','highRepOverlap'
filter_perts = "highRepOverlap"

################################################
pertColName = "PERT"
profileLevel = "treatment"
#'replicate'  or  'treatment'
if filter_perts:
    f = "filt"
else:
    f = ""

(
    mergProf_treatLevel_LI,
    cp_features_LI,
    l1k_features_LI,
) = read_paired_treatment_level_profiles(
    procProf_dir, "LINCS", profileType, filter_perts, 1
)

(
    mergProf_treatLevel_LU,
    cp_features_LU,
    l1k_features_LU,
) = read_paired_treatment_level_profiles(
    procProf_dir, "CDRP-bio", profileType, filter_perts, 1
)


In [ ]:
cp_features_unionFs = list(set(cp_features_LU).union(set(cp_features_LI)))
l1k_features_unionFs = list(set(l1k_features_LU).union(set(l1k_features_LI)))


In [ ]:
%%capture
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor

# cp_features=list(set(cp_features_LU) & set(cp_features_LI))
# l1k_features=list(set(l1k_features_LU) & set(l1k_features_LI))
# profileType='normalized'

cp_features=list(set(cp_features_LU) & set(cp_features_LI) & set(cp_features_unionFs))
l1k_features=list(set(l1k_features_LU) & set(l1k_features_LI)& set(l1k_features_unionFs))
profileType='normalized_variable_selected_union'


####################
l1k_LU=mergProf_treatLevel_LU[[pertColName]+l1k_features]
cp_LU=mergProf_treatLevel_LU[[pertColName]+cp_features]

l1k_LI=mergProf_treatLevel_LI[[pertColName]+l1k_features]
cp_LI=mergProf_treatLevel_LI[[pertColName]+cp_features]

####################


l1k_scaled_LU=l1k_LU.copy();l1k_scaled_LI=l1k_LI.copy()
l1k_scaled_LU[l1k_features] = preprocessing.StandardScaler().fit_transform(l1k_LU[l1k_features].values)
l1k_scaled_LI[l1k_features] = preprocessing.StandardScaler().fit_transform(l1k_LI[l1k_features].values)

cp_scaled_LU=cp_LU.copy();cp_scaled_LI=cp_LI.copy()
cp_scaled_LU[cp_features] = preprocessing.StandardScaler().fit_transform(cp_LU[cp_features].values.astype('float64'))
cp_scaled_LI[cp_features] = preprocessing.StandardScaler().fit_transform(cp_LI[cp_features].values.astype('float64'))

fold1=[[cp_scaled_LU,l1k_scaled_LU],[cp_scaled_LI,l1k_scaled_LI]]
fold2=[[cp_scaled_LI,l1k_scaled_LI],[cp_scaled_LU,l1k_scaled_LU]]


pred_df=pd.DataFrame(index=range(2),columns=l1k_features)
pred_df_rand=pd.DataFrame(index=range(2),columns=l1k_features)

# for dt in [fold1,fold2]:
for n,dt in zip([0,1],[fold1,fold2]):
    
    for model in ["Lasso"]:#["Lasso","MLP","Ridge"]:    

        ##############################
        ii=0
        for l in l1k_features:
            ii+=1
            print(ii)

            X_train, X_test = dt[0][0][cp_features].values, dt[1][0][cp_features].values
            y_train, y_test = dt[0][1][l].values, dt[1][1][l]#.values

            if model=="Lasso":
    #             scores,scores_rand=lasso_cv_plus_model_selection(cp[cp_features],l1k[l],k_fold,group_labels)        
    # #                 scores,scores_rand=lasso_cv(cp[cp_features],l1k[l],k_fold,group_labels)
                alphas1 = np.linspace(0, 0.2, 20)
                alphas2 = np.linspace(0.2, 0.5, 10)[1:]
                alphas=np.concatenate((alphas1,alphas2))
            #     alphas = np.logspace(-4, -0.5, 30)
                lasso_cv = linear_model.LassoCV(alphas=alphas, random_state=0, max_iter=1000,selection='random')

                lasso_cv.fit(X_train, y_train)  
                r2_score=lasso_cv.score(X_test, y_test.values) 
                r2_rand=lasso_cv.score(X_test, y_test.sample(frac = 1).values)

            if model=="Ridge":
    #             scores,scores_rand=lasso_cv_plus_model_selection(cp[cp_features],l1k[l],k_fold,group_labels)        
    # #                 scores,scores_rand=lasso_cv(cp[cp_features],l1k[l],k_fold,group_labels)
                alphas = np.linspace(0.1, 0.2, 20)
            #     alphas = np.logspace(-4, -0.5, 30)
                ridge_cv = linear_model.RidgeCV(alphas=alphas)

                ridge_cv.fit(X_train, y_train)  
                r2_score=ridge_cv.score(X_test, y_test.values) 
                r2_rand=ridge_cv.score(X_test, y_test.sample(frac = 1).values)
                
            elif model=="MLP":
#                 scores,scores_rand=MLP_cv_plus_model_selection(cp[cp_features],l1k[l],k_fold,group_labels)
    #                 scores,scores_rand=MLP_cv(cp[cp_features],l1k[l],k_fold,group_labels)  
                mlp_gs = MLPRegressor(activation='logistic',max_iter=500)
                parameter_space = {
                    'hidden_layer_sizes': [(50,),(10,30,10),(50,10),(50,10,10)],
                    'alpha': [0.0001, 0.05,0.01,0.2],
                    'early_stopping':[True,False]
                }

                clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=2)
                clf.fit(X_train, y_train)  
                r2_score=clf.score(X_test, y_test.values)    
                r2_rand=clf.score(X_test, y_test.sample(frac = 1).values)
    
            pred_df.loc[n,l]=r2_score
            pred_df_rand.loc[n,l]=r2_rand

    
########################### mapping prob_ids to genes names    
meta=pd.read_csv("../affy_probe_gene_mapping.txt",delimiter="\t",header=None, names=["probe_id", "gene"])
meta_gene_probID=meta.set_index('probe_id')
d = dict(zip(meta_gene_probID.index, meta_gene_probID['gene']))
pred_df = pred_df.rename(columns=d)    
pred_df_rand = pred_df_rand.rename(columns=d)  

pred_df.loc[0,'DT']='CDRPbio-LINCS'
pred_df.loc[1,'DT']='LINCS-CDRPbio'

pred_df_rand.loc[0,'DT']='CDRPbio-LINCS'
pred_df_rand.loc[1,'DT']='LINCS-CDRPbio'

meltedPredDF=pd.melt(pred_df,id_vars='DT').rename(columns={'variable':'lmGens','value':'pred score'})
meltedPredDF_rand=pd.melt(pred_df_rand,id_vars='DT').rename(columns={'variable':'lmGens','value':'pred score'})
meltedPredDF['d']="n-folds"
meltedPredDF_rand['d']="random"

filename=results_dir+'/SingleGenePred/scores_cross_dts_CD_LI.xlsx'

profTypeAbbrev=''.join([s[0] for s in profileType.split('_')])

saveAsNewSheetToExistingFile(filename,pd.concat([meltedPredDF,meltedPredDF_rand],ignore_index=True),\
                             model+'-'+profTypeAbbrev+'-'+f+'-ht')
        